In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms

In [ ]:
# GPU 장치 사용 설정
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")

### ImageNet 에 정의된 클래스 정보 가져오기

In [ ]:
from urllib.request import urlretrieve
import json

imagenet_json, _ = urlretrieve("http://www.anishathalye.com/media/2017/07/25/imagenet.json")
with open(imagenet_json) as f:
  imagenet_labels = json.load(f)

In [ ]:
print(imagenet_labels[18])

magpie


ResNet 은 일반적으로 이미지에 대하여 Resize, CenterCrop ToTensor() 와 입력 데이터 정규화를 사용하는 모델이다.

In [ ]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

In [ ]:
import matplotlib.pyplot as plt
import PIL

In [ ]:
# 특정한 경로에서 이미지를 가져와 torch.Tensor 로 변환하는 함수
def image_loader(path):
  image = PIL.Image.open(path)
  # 전처리 이후, 네트워크 입력에 들어갈 이미지에 배치 목적의 차원 추가
  image = preprocess(image).unsqueeze(0)

pytorch 의 unsqueeze() 함수는 지정한 dimensino 자리에 size 가 1인 빈 공간을 채워주면서 차원을 확장한다. 

https://jimmy-ai.tistory.com/110

In [ ]:
# torch.Tensor 형태의 함수를 이미지로 다시 변환하는 함수
def imshow(tensor):
  # matplotlib 는 CPU 기반이므로 CPU로 옮기기
  image = tensor.cpu().clone()
  # torch.Tensor 에서 사용되는 배치 목적의 차원 제거
  image = image.squeeze(0)
  # PIL 객체로 변경
  image = transforms.ToPILimage()(image)
  plt.imshow(image)

### 사전 학습된 모델을 불러와 사용해보기

In [ ]:
# 입력 데이터 정규화를 위한 클래스 정의
class Normalize(nn.Module):
  def __init__(self, mean, std):
    super(Normalize, self).__init__()
    self.register_buffer('mean', torch.Tensor(mean))
    self.register_buffer('std', torch.Tensor(std))

  def forward(self, input):
    mean = self.mean.reshape(1, 3, 1, 1)
    std = self.std.reshape(1, 3, 1, 1)
    return (input - mean) / std

위에서 사용된 register_buffer 는 파라미터가 아닌 단순 매개변수로서 사용하기 위한 Module 클래스의 기능이다. GPU에서 사용가능하지만, 최적화에는 관여하지 않는다.

https://aigong.tistory.com/429#torch.nn.Module.register_buffer(name,_tensor,_persistent)

In [ ]:
model = nn.Sequential(
    # 기본적인 ResNet18과 동일한 동작을 위하여 정규화 레이어 추가
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=True)
).to(device).eval()

Downloading: "https://github.com/pytorch/vision/archive/v0.6.0.zip" to /root/.cache/torch/hub/v0.6.0.zip
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

ImageNet 데이터를 다운로드 받지 않은 상태이기 때문에 일단, image 라는 변수로 데이터 하나를 가져왔다고 가정한다.

In [ ]:
# 기본적인 이미질르 실제 모델에 넣어 결과 확인
outputs = model(image)
# 확률을 계산하기 위해 소프트맥스 함수 통과
percentages = torch.nn.functional.softmax(outputs, dim=1)[0] * 100
# 가장 높은 값을 가지는 5개의 인덱스를 하나씩 확인하며
for i in outputs[0].topk(5)[1]:
  # 높은 값을 가지는 순서대로 인덱스에 해당하는 클래스 이름과, 확률값 출력
  print(f"인덱스 : {i.item()} / 클래스명 : {imagenet_labels[i]} / 확률 : {round(percentages[i].item(), 4)}%")